# Import relevant libraries/modules etc

In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio, xarray_to_rasterio_by_band
import rasterstats

In [2]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar
pbar = ProgressBar()
pbar.register()

# Read NetCDF files in to create dataset with all data in

In [3]:
PM25 = xr.open_mfdataset(r'C:\MAIACData\nc_monthly_daily\*PM25.nc')['data']

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [4]:
PM25

<xarray.DataArray 'data' (time: 5191, y: 1162, x: 1240)>
dask.array<concate..., shape=(5191, 1162, 1240), dtype=float32, chunksize=(30, 1162, 1240)>
Coordinates:
  * x        (x) float64 -9.476e+05 -9.464e+05 -9.451e+05 -9.439e+05 ...
  * y        (y) float64 1.429e+06 1.428e+06 1.427e+06 1.426e+06 1.424e+06 ...
  * time     (time) datetime64[ns] 2000-10-01 2000-10-02 2000-10-03 ...
Attributes:
    affine: [ -9.47639631e+05   1.25654304e+03   0.00000000e+00   1.42927781e+06
   0.00000000e+00  -1.25654304e+03]
    crs: +init=epsg:27700

# Extract data over one pixel

In [52]:
a = PM25.attrs['affine']
a = rasterio.Affine.from_gdal(*a)
~a * (382611, 80604)

(1058.659022276559, 1073.3208212376835)

In [9]:
PM25 = PM25.isel(time=np.argsort(PM25.time))

After2009 = PM25.sel(time=slice('2009', '2016'))

monthly_data = After2009.resample('M', dim='time', how='mean', keep_attrs=True)

In [53]:
ts = PM25.isel(x=1058, y=1073).load()

[########################################] | 100% Completed |  8min  5.7s


In [54]:
ts

<xarray.DataArray 'data' (time: 5191)>
array([ 21.28530693,          nan,          nan, ...,          nan,
                nan,          nan], dtype=float32)
Coordinates:
    x        float64 3.818e+05
    y        float64 8.101e+04
  * time     (time) datetime64[ns] 2000-02-25 2000-02-26 2000-02-27 ...
Attributes:
    affine: [ -9.47639631e+05   1.25654304e+03   0.00000000e+00   1.42927781e+06
   0.00000000e+00  -1.25654304e+03]
    crs: +init=epsg:27700

In [55]:
result = ts.to_dataframe()

In [56]:
result.dropna()

,x,y,data
time,,,
2000-02-25,381782.910143,81007.125695,21.285307
2000-03-04,381782.910143,81007.125695,23.822201
2000-03-12,381782.910143,81007.125695,18.680717
2000-03-19,381782.910143,81007.125695,19.966087
2000-04-08,381782.910143,81007.125695,12.970246
2000-04-10,381782.910143,81007.125695,9.933137
2000-04-29,381782.910143,81007.125695,4.823646
2000-04-30,381782.910143,81007.125695,3.930378
2000-05-01,381782.910143,81007.125695,7.974380


In [57]:
result.to_csv(r'D:\Annies_Dissertation\Analysis\Timeseries\Westlulworth.csv')

## Merging timeseries data so can create a graph

In [26]:
from dateutil.parser import parse

In [61]:
Unisoton = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Timeseries\UniSoton.csv', parse_dates=['time'])

In [62]:
Unisoton[:10]

,time,x,y,data
0,2000-02-25,442096.976259,116190.330929,10.683363
1,2000-02-26,442096.976259,116190.330929,NaN
2,2000-02-27,442096.976259,116190.330929,NaN
3,2000-02-28,442096.976259,116190.330929,16.978916
4,2000-02-29,442096.976259,116190.330929,NaN
5,2000-03-01,442096.976259,116190.330929,NaN
6,2000-03-02,442096.976259,116190.330929,NaN
7,2000-03-03,442096.976259,116190.330929,16.339218
8,2000-03-04,442096.976259,116190.330929,16.877872
9,2000-03-05,442096.976259,116190.330929,NaN


In [63]:
Unisoton = Unisoton.set_index('time').rename(columns={'data':'Unisoton_data'})

In [64]:
Bolderwood = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Timeseries\Bolderwood.csv', parse_dates=['time'])

In [65]:
Bolderwood[:10]

,time,x,y,data
0,2000-02-25,423248.830598,108651.072665,4.444452
1,2000-02-26,423248.830598,108651.072665,15.886553
2,2000-02-27,423248.830598,108651.072665,NaN
3,2000-02-28,423248.830598,108651.072665,15.319175
4,2000-02-29,423248.830598,108651.072665,NaN
5,2000-03-01,423248.830598,108651.072665,NaN
6,2000-03-02,423248.830598,108651.072665,NaN
7,2000-03-03,423248.830598,108651.072665,NaN
8,2000-03-04,423248.830598,108651.072665,NaN
9,2000-03-05,423248.830598,108651.072665,15.506694


In [66]:
Bolderwood = Bolderwood.set_index('time').rename(columns={'data':'Bolderwood_data'})

In [67]:
result = pd.merge(Unisoton, Bolderwood, left_index=True, right_index=True)

In [70]:
result[:10]

,Unisoton_data,Bolderwood_data
time,,
2000-02-25,10.683363,4.444452
2000-02-26,NaN,15.886553
2000-02-27,NaN,NaN
2000-02-28,16.978916,15.319175
2000-02-29,NaN,NaN
2000-03-01,NaN,NaN
2000-03-02,NaN,NaN
2000-03-03,16.339218,NaN
2000-03-04,16.877872,NaN


In [71]:
Westlulworth = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Timeseries\Westlulworth.csv', parse_dates=['time'])

In [74]:
Westlulworth[:10]

,x,y,Westlulworth_data
time,,,
2000-02-25,381782.910143,81007.125695,21.285307
2000-02-26,381782.910143,81007.125695,NaN
2000-02-27,381782.910143,81007.125695,NaN
2000-02-28,381782.910143,81007.125695,NaN
2000-02-29,381782.910143,81007.125695,NaN
2000-03-01,381782.910143,81007.125695,NaN
2000-03-02,381782.910143,81007.125695,NaN
2000-03-03,381782.910143,81007.125695,NaN
2000-03-04,381782.910143,81007.125695,23.822201


In [73]:
Westlulworth = Westlulworth.set_index('time').rename(columns={'data':'Westlulworth_data'})

In [75]:
result_3 = pd.merge(result, Westlulworth, left_index=True, right_index=True)

In [80]:
result_3[:10]

,Unisoton_data,Bolderwood_data,Westlulworth_data
time,,,
2000-02-25,10.683363,4.444452,21.285307
2000-02-26,NaN,15.886553,NaN
2000-02-27,NaN,NaN,NaN
2000-02-28,16.978916,15.319175,NaN
2000-02-29,NaN,NaN,NaN
2000-03-01,NaN,NaN,NaN
2000-03-02,NaN,NaN,NaN
2000-03-03,16.339218,NaN,NaN
2000-03-04,16.877872,NaN,23.822201


In [79]:
result_3 = result_3[['Unisoton_data', 'Bolderwood_data', 'Westlulworth_data']]

In [81]:
result_3.dropna().to_csv(r'D:\Annies_Dissertation\Analysis\Timeseries\Timeseries.csv')

In [50]:
result[:10]

,Unisoton_data,Bolderwood_data
time,,
2000-02-25,10.683363,4.444452
2000-02-26,NaN,15.886553
2000-02-27,NaN,NaN
2000-02-28,16.978916,15.319175
2000-02-29,NaN,NaN
2000-03-01,NaN,NaN
2000-03-02,NaN,NaN
2000-03-03,16.339218,NaN
2000-03-04,16.877872,NaN


# Group data
- merge multiple images per day using the maximum AOT estimates

In [ ]:
PM25 = PM25.isel(time=np.argsort(PM25.time))

In [ ]:
PM25.time

In [ ]:
m = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
m = m.load()

In [ ]:
figure(figsize=(20, 8))
m.plot()

In [ ]:
from dask.dot import dot_graph

In [ ]:
##dot_graph(, format='svg')

In [ ]:
from dask.dot import to_graphviz

In [ ]:
dot_graph(m.data.dask)

In [ ]:
dot_graph(m.data.dask, filename='robin2.pdf', format='pdf')

In [ ]:
reordered_PM25 = PM25.isel(time=np.argsort(PM25.time))

In [ ]:
Daily_PM25 = reordered_PM25.resample('D', dim='time', how='max')
Daily_PM25 = Daily_PM25.dropna(dim='time', how='all')

In [ ]:
Daily_PM25.time

In [ ]:
Daily_PM25.attrs

# Analysis
## overall average of every pixel

The next four cells were added by Robin for profiling - ignore them for the moment :P

In [ ]:
om = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
    om.load()

In [ ]:
from dask.diagnostics import visualize
visualize([prof, rprof], show=True)

In [ ]:
overall_mean = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
overall_mean.attrs

In [ ]:
with ProgressBar():
    res = overall_mean.load()

In [ ]:
xarray_to_rasterio(overall_mean, r'D:\Annies_Dissertation\Analysis\overall_mean.tif')

## Averages
### Monthly

In [ ]:
monthly_mean = PM25.groupby('time.month').mean(dim='time', keep_attrs=True)

In [ ]:
monthly_mean.coords

In [ ]:
monthly_mean.attrs

In [ ]:
figure(figsize=(20, 12))
monthly.plot(col='month', robust=True)

In [ ]:
xarray_to_rasterio_by_band(monthly_mean, r'D:\Annies_Dissertation\Analysis\monthly\monthly_mean_', dim='month')

### Seasonal

In [ ]:
seasonal_mean = PM25.groupby('time.season').mean(dim='time', keep_attrs=True)

In [ ]:
seasonal_mean.coords

In [ ]:
figure(figsize=(20, 12))
seasonal_mean.plot(col='season', robust=True)

In [ ]:
xarray_to_rasterio_by_band(seasonal_mean, r'D:\Annies_Dissertation\Analysis\seasonal\seasonal_mean_', dim='season')

### Yearly

In [ ]:
yearly_mean = PM25.groupby('time.year').mean(dim='time', keep_attrs=True)

In [ ]:
yearly_mean.coords

In [ ]:
figure(figsize=(20, 12))
yearly_mean.plot(col='year', robust=True)

In [ ]:
xarray_to_rasterio_by_band(yearly_mean, r'D:\Annies_Dissertation\Analysis\yearly\yearly_mean_', dim='year')

## Standard deviation of AP for each pixel over time- not currently what i want to do!

In [ ]:
std = PM25.groupby('time.year').std(dim = 'time', keep_attrs=True)

In [ ]:
std = PM25.std(dim = 'time', keep_attrs=True)

In [ ]:
std.coords

In [ ]:
figure(figsize=(20, 12))
std.plot(col='year', robust=True)

In [ ]:
xarray_to_rasterio(std, r'D:\Annies_Dissertation\Analysis\overall_std.tif')

# Count of values that aren't missing

In [ ]:
observations = PM25.count(dim='time', keep_attrs=True)

In [ ]:
xarray_to_rasterio(observations, r'D:\Annies_Dissertation\Analysis\observations_count.tif')

In [ ]:
count_histo = observations.plot.hist()
savefig(r'D:\Annies_Dissertation\Analysis\count_histogram.jpeg')

In [ ]:
figure(figsize=(20, 12))
observations.plot(robust=True)

# Regression
- linear regression for each pixel (Rabernat code)

In [ ]:
import xarray as xr
import numpy as np

# create an example dataset
# da = xr.DataArray(np.random.rand(10,30,40), dims=['time', 'lat', 'lon'])

# define a function to compute a linear trend of a timeseries
def linear_trend(x):
    pf = np.polyfit(x.time, x, 1)
    # we need to return a dataarray or else xarray's groupby won't be happy
    return xr.DataArray(pf[0])

In [ ]:
import xarray as xr
import numpy as np
from scipy import stats

# create an example dataset
# da = xr.DataArray(np.random.rand(10,30,40), dims=['time', 'lat', 'lon'])

# define a function to compute a linear trend of a timeseries
def linear_trend(x):
    pf = stats.linregress(x.time, x)
    # we need to return a dataarray or else xarray's groupby won't be happy
    return xr.DataArray(pf[0])

**ROBIN ADDED THIS BIT BELOW**
To subset the data to deal with the silly dask errors...

In [ ]:
# Work on a subset of the data due to dask errors!
subset = PM25.isel(x=slice(900, None), y=slice(900, None))

In [ ]:
# stack lat and lon into a single dimension called allpoints
stacked = subset.stack(allpoints=['y','x'])
# apply the function over allpoints to calculate the trend at each point
trend = stacked.groupby('allpoints').apply(linear_trend)
# unstack back to lat lon coordinates
trend_unstacked = trend.unstack('allpoints')

In [ ]:
trend_unstacked

In [ ]:
After2009 = PM25.sel(time=slice('2009', '2016'))

In [ ]:
After2009.time.min(), After2009.time.max()

# Zonal Statistics

In [ ]:
from rasterstats import zonal_stats
zonal_stats(r"D:\Annies_Dissertation\Data\LSOA\LSOA.shp", 
            r"D:\Annies_Dissertation\MAIAC_Test\subset\Projected\2014\MAIACAAOT.h00v01.20140041325.hdf_PM25.tif",
            stats="count min mean max median")

In [ ]:
from rasterstats import zonal_stats
zonal_stats(r"D:\Annies_Dissertation\Data\LSOA\LSOA.shp", 
            r"D:\Annies_Dissertation\MAIAC_Test\subset\Projected\2014\MAIACAAOT.h00v01.20140041325.hdf_PM25.tif",
            stats="count min mean max median", geojson_out=True)